In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

In [2]:
# Load the data
with open('/content/drive/MyDrive/Colab Notebooks/modified_poems.txt', 'r') as file:
    text = file.read().lower()

# Tokenize the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

# Convert text to sequences of integer values
sequences = tokenizer.texts_to_sequences([text])[0]

# Prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []
for i in range(0, len(sequences) - seq_length, 1):
    seq_in = sequences[i:i + seq_length]
    seq_out = sequences[i + seq_length]
    dataX.append([int(char) for char in seq_in])
    dataY.append(int(seq_out))

# Total number of unique words
num_words = len(tokenizer.word_index) + 1

# Prepare the dataset of input to output pairs encoded as integers
x = np.array(dataX)
y = to_categorical(dataY, num_classes=num_words)


In [30]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=256, input_length=seq_length))
model.add(LSTM(440, return_sequences=True))
model.add(LSTM(440))
model.add(Dense(num_words, activation='softmax'))


# Compile the model
optimizer = RMSprop(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)



# Prepare the callback
early_stopping = EarlyStopping(monitor='loss', patience=4)  # Stop if loss doesn't improve for 4 consecutive epochs

# Add it to the `fit` method
model.fit(x, y, batch_size=30, epochs=120, callbacks=[early_stopping], validation_split=0.1)


model.save('poem4.h5')


Epoch 1/120
760/760 [==============================] - 14s 12ms/step - loss: 6.1141 - val_loss: 6.1270
Epoch 2/120
760/760 [==============================] - 9s 12ms/step - loss: 5.6447 - val_loss: 5.9636
Epoch 3/120
760/760 [==============================] - 9s 12ms/step - loss: 5.3580 - val_loss: 5.8496
Epoch 4/120
760/760 [==============================] - 9s 11ms/step - loss: 5.0795 - val_loss: 5.7892
Epoch 5/120
760/760 [==============================] - 9s 11ms/step - loss: 4.9242 - val_loss: 5.7929
Epoch 6/120
760/760 [==============================] - 9s 12ms/step - loss: 4.7583 - val_loss: 5.7435
Epoch 7/120
760/760 [==============================] - 9s 11ms/step - loss: 4.5750 - val_loss: 5.7338
Epoch 8/120
760/760 [==============================] - 9s 12ms/step - loss: 4.3770 - val_loss: 5.7047
Epoch 9/120
760/760 [==============================] - 9s 12ms/step - loss: 4.1835 - val_loss: 5.5011
Epoch 10/120
760/760 [==============================] - 9s 11ms/step - loss: 3.97

In [31]:
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_seq(model, tokenizer, seed_text, seq_length, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict(encoded, verbose=0).argmax(axis=-1)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

def format_poem(poem, words_per_line=5):
    words = poem.split()
    lines = [' '.join(words[i:i+words_per_line]) for i in range(0, len(words), words_per_line)]
    return '\n'.join(lines)

# load the model
model = load_model('/content/poem5.h5')

# specify the seed text and the length of the generated sequence
seed_text = 'Uplifting '
seq_length = 40  # should be the same as seq_length during training
n_words = 25  # number of words to generate

# generate new text
generated = generate_seq(model, tokenizer, seed_text, seq_length, n_words)

# format the generated text
formatted_poem = format_poem(generated)

print(formatted_poem)



where despair as their their
hand in war's realm sacrificial
love ripples unspoken life's place
together as strength we mend
uplifting spirits like hearts them


poem.h5, epochs=12, bsize=50, Units=500
learning rate=0.01

output:
and light and peace and
peace and peace and peace
and peace and peace and
peace and peace and peace
and peace

poem.h5, epochs complete=35, bsize=50, Units = 300 learning rate=0.01

output with seed text "India":
free whispers on hope's presence
is a melody sung by
a beacon of light in
the darkest night steady calm
and serene a promise of

poem2.h5, epochs completed=35, bsize=50, Units = 400 learning rate=0.001
output with seed_text "India"
Epoch 70/75
456/456 [==============================] - 4s 9ms/step - loss: 0.2661 - val_loss: 9.1319
Epoch 71/75
456/456 [==============================] - 4s 9ms/step - loss: 0.2507 - val_loss: 9.1534
Epoch 72/75
456/456 [==============================] - 4s 9ms/step - loss: 0.2405 - val_loss: 9.1511
Epoch 73/75
456/456 [==============================] - 4s 9ms/step - loss: 0.2296 - val_loss: 9.2486
Epoch 74/75
456/456 [==============================] - 4s 9ms/step - loss: 0.2239 - val_loss: 9.3994
Epoch 75/75
456/456 [==============================] - 4s 9ms/step - loss: 0.2078 - val_loss: 9.4394

we can see that the loss is getting better

poem3.h5, epochs completed=92, bsize=40, Units = 440 learning rate=0.001
Epoch 92/120
570/570 [==============================] - 5s 9ms/step - loss: 0.0978 - val_loss: 11.7174
output with seed_text "Uplifting Souls"

as radiant power day in
the realm of sunshine dreams
find their way a symphony
of possibilities where dreams can
near it fuels determination igniting